In [91]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, log_loss
import operator
import json
from IPython import display
import os
import warnings
import itertools
np.random.seed(0)
warnings.filterwarnings("ignore")
THRESHOLD = 4

Task: To predict whether the user likes the mobile phone or not. <br>
Assumption: If the average rating of mobile >= threshold, then the user likes it, otherwise not.

<b>Missing values:</b><br>
'Also Known As'(459),'Applications'(421),'Audio Features'(437),'Bezel-less display'(266),'Browser'(449),'Build Material'(338),'Co-Processor'(451),'Display Colour'(457),'Mobile High-Definition Link(MHL)'(472),'Music'(447)
'Email','Fingerprint Sensor Position'(174),'Games'(446),'HDMI'(454),'Heart Rate Monitor'(467),'IRIS Scanner'(467),
'Optical Image Stabilisation'(219),'Other Facilities'(444),'Phone Book'(444),'Physical Aperture'(87),'Quick Charging'(122),'Ring Tone'(444),'Ruggedness'(430),SAR Value(315),'SIM 3'(472),'SMS'(470)', 'Screen Protection'(229),'Screen to Body Ratio (claimed by the brand)'(428),'Sensor'(242),'Software Based Aperture'(473),
'Special Features'(459),'Standby time'(334),'Stylus'(473),'TalkTime'(259), 'USB Type-C'(374),'Video Player'(456),
'Video Recording Features'(458),'Waterproof'(398),'Wireless Charging','USB OTG Support'(159), 'Video ,'Recording'(113),'Java'(471),'Browser'(448)

<b>Very low variance:</b><br>
'Architecture'(most entries are 64-bit),'Audio Jack','GPS','Loudspeaker','Network','Network Support','Other Sensors'(28),'SIM Size', 'VoLTE'


<b>Multivalued:</b><br>
'Colours','Custom UI','Model'(1),'Other Sensors','Launch Date'

<b>Not important:</b><br>
'Bluetooth', 'Settings'(75),'Wi-Fi','Wi-Fi Features'

<b>Doubtful:</b><br>
'Aspect Ratio','Autofocus','Brand','Camera Features','Fingerprint Sensor'(very few entries are missing),
'Fingerprint Sensor Position', 'Graphics'(multivalued),'Image resolution'(multivalued),'SIM Size','Sim Slot(s)', 'User Available Storage', 'SIM 1', 'SIM 2','Shooting Modes', 'Touch Screen'(24), 'USB Connectivity'
    
<b>To check:</b><br>
'Display Type','Expandable Memory','FM Radio'

<b>High Correlation with other features</b><br>
'SIM Slot(s)' high correlation with SIM1
'Weight' has high high correlation with capacity , screen-to-body ratio
'Height' - screen size is also there
    
<b>Given a mobile, we can't directly get these features</b><br>
'Rating Count', 'Review Count'

<b>Keeping:</b><br>
'Capacity','Flash'(17),'Height'(22),'Internal Memory'(20, require cleaning),'Operating System'(25, require cleaning), 'Pixel Density'(1, clean it),'Processor'(22, clean it), 'RAM'(17, clean), 'Rating','Resolution'(cleaning), 'Screen Resolution','Screen Size', 'Thickness'(22), 'Type','User Replaceable','Weight'(cleaning),'Sim Size'(), 'Other Sensors'(28), 'Screen to Body Ratio (calculated)','Width',


In [92]:
# read data from file
# train = pd.read_csv("../input/train.csv") 
# test = pd.read_csv("../input/test.csv")
train = pd.read_csv("train.csv") 
test = pd.read_csv("test.csv")


# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 355
Number of features in train: 99
Number of data points in test: 119
Number of features in test: 98


In [93]:
def data_clean(data):
    
    # Let's first remove all missing value features
    columns_to_remove = ['Also Known As','Applications','Audio Features','Bezel-less display'
                         'Browser','Build Material','Co-Processor','Browser'
                         'Display Colour','Mobile High-Definition Link(MHL)',
                         'Music', 'Email','Fingerprint Sensor Position',
                         'Games','HDMI','Heart Rate Monitor','IRIS Scanner', 
                         'Optical Image Stabilisation','Other Facilities',
                         'Phone Book','Physical Aperture','Quick Charging',
                         'Ring Tone','Ruggedness','SAR Value','SIM 3','SMS',
                         'Screen Protection','Screen to Body Ratio (claimed by the brand)',
                         'Sensor','Software Based Aperture', 'Special Features',
                         'Standby time','Stylus','TalkTime', 'USB Type-C',
                         'Video Player', 'Video Recording Features','Waterproof',
                         'Wireless Charging','USB OTG Support', 'Video Recording','Java']

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    #Features having very low variance 
    columns_to_remove = ['Architecture','Audio Jack','GPS','Loudspeaker','Network','Network Support','VoLTE']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    # Multivalued:
    columns_to_remove = ['Architecture','Launch Date','Audio Jack','GPS','Loudspeaker','Network','Network Support','VoLTE', 'Custom UI']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    # Not much important
    columns_to_remove = ['Bluetooth', 'Settings','Wi-Fi','Wi-Fi Features']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]
    
    return data

# Removing features

In [94]:
train = data_clean(train)
test = data_clean(test)

removing all those data points in which more than 15 features are missing 

In [95]:
train = train[(train.isnull().sum(axis=1) <= 15)]
# You shouldn't remove data points from test set
#test = test[(test.isnull().sum(axis=1) <= 15)]

In [96]:
# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 341
Number of features in train: 47
Number of data points in test: 119
Number of features in test: 46


# Filling Missing values

In [97]:
def for_integer(test):
    try:
        test = test.strip()
        return int(test.split(' ')[0])
    except IOError:
           pass
    except ValueError:
        pass
    except:
        pass

def for_string(test):
    try:
        test = test.strip()
        return (test.split(' ')[0])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass

def for_float(test):
    try:
        test = test.strip()
        return float(test.split(' ')[0])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass
def find_freq(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[2][0] == '(':
            return float(test[2][1:])
        return float(test[2])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass

    
def for_Internal_Memory(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[1] == 'GB':
            return int(test[0])
        if test[1] == 'MB':
#             print("here")
            return (int(test[0]) * 0.001)
    except IOError:
           pass
    except ValueError:
        pass
    except:
        pass
    
def find_freq(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[2][0] == '(':
            return float(test[2][1:])
        return float(test[2])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass


In [98]:
def data_clean_2(x):
    data = x.copy()
    
    data['Capacity'] = data['Capacity'].apply(for_integer)

    data['Height'] = data['Height'].apply(for_float)
    data['Height'] = data['Height'].fillna(data['Height'].mean())

    data['Internal Memory'] = data['Internal Memory'].apply(for_Internal_Memory)

    data['Pixel Density'] = data['Pixel Density'].apply(for_integer)

    data['Internal Memory'] = data['Internal Memory'].fillna(data['Internal Memory'].median())
    data['Internal Memory'] = data['Internal Memory'].astype(int)

    data['RAM'] = data['RAM'].apply(for_integer)
    data['RAM'] = data['RAM'].fillna(data['RAM'].median())
    data['RAM'] = data['RAM'].astype(int)

    data['Resolution'] = data['Resolution'].apply(for_integer)
    data['Resolution'] = data['Resolution'].fillna(data['Resolution'].median())
    data['Resolution'] = data['Resolution'].astype(int)

    data['Screen Size'] = data['Screen Size'].apply(for_float)

    data['Thickness'] = data['Thickness'].apply(for_float)
    data['Thickness'] = data['Thickness'].fillna(data['Thickness'].mean())
    data['Thickness'] = data['Thickness'].round(2)

    data['Type'] = data['Type'].fillna('Li-Polymer')

    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].apply(for_float)
    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].fillna(data['Screen to Body Ratio (calculated)'].mean())
    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].round(2)

    data['Width'] = data['Width'].apply(for_float)
    data['Width'] = data['Width'].fillna(data['Width'].mean())
    data['Width'] = data['Width'].round(2)

    data['Flash'][data['Flash'].isna() == True] = "Other"

    data['User Replaceable'][data['User Replaceable'].isna() == True] = "Other"

    data['Num_cores'] = data['Processor'].apply(for_string)
    data['Num_cores'][data['Num_cores'].isna() == True] = "Other"


    data['Processor_frequency'] = data['Processor'].apply(find_freq)
    #because there is one entry with 208MHz values, to convert it to GHz
    data['Processor_frequency'][data['Processor_frequency'] > 200] = 0.208
    data['Processor_frequency'] = data['Processor_frequency'].fillna(data['Processor_frequency'].mean())
    data['Processor_frequency'] = data['Processor_frequency'].round(2)

    data['Camera Features'][data['Camera Features'].isna() == True] = "Other"

    #simplifyig Operating System to os_name for simplicity
    data['os_name'] = data['Operating System'].apply(for_string)
    data['os_name'][data['os_name'].isna() == True] = "Other"

    data['Sim1'] = data['SIM 1'].apply(for_string)

    data['SIM Size'][data['SIM Size'].isna() == True] = "Other"

    data['Image Resolution'][data['Image Resolution'].isna() == True] = "Other"

    data['Fingerprint Sensor'][data['Fingerprint Sensor'].isna() == True] = "Other"

    data['Expandable Memory'][data['Expandable Memory'].isna() == True] = "No"

    data['Weight'] = data['Weight'].apply(for_integer)
    data['Weight'] = data['Weight'].fillna(data['Weight'].mean())
    data['Weight'] = data['Weight'].astype(int)

    data['SIM 2'] = data['SIM 2'].apply(for_string)
    data['SIM 2'][data['SIM 2'].isna() == True] = "Other"
    
    return data

In [99]:
train = data_clean_2(train)
test = data_clean_2(test)

# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 341
Number of features in train: 51
Number of data points in test: 119
Number of features in test: 50


Not very important feature

In [100]:
def data_clean_3(x):
    
    data = x.copy()

    columns_to_remove = ['User Available Storage','SIM Size','Chipset','Processor','Autofocus','Aspect Ratio','Touch Screen',
                        'Bezel-less display','Operating System','SIM 1','USB Connectivity','Other Sensors','Graphics','FM Radio',
                        'NFC','Shooting Modes','Browser','Display Colour' ]

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]


    columns_to_remove = [ 'Screen Resolution','User Replaceable','Camera Features',
                        'Thickness', 'Display Type']

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]


    columns_to_remove = ['Fingerprint Sensor', 'Flash', 'Rating Count', 'Review Count','Image Resolution','Type','Expandable Memory',\
                        'Colours','Width','Model']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    return data

In [101]:
train = data_clean_3(train)
test = data_clean_3(test)

# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 341
Number of features in train: 18
Number of data points in test: 119
Number of features in test: 17


In [102]:
# one hot encoding

train_ids = train['PhoneId']
test_ids = test['PhoneId']

cols = list(test.columns)
cols.remove('PhoneId')
cols.insert(0, 'PhoneId')

combined = pd.concat([train.drop('Rating', axis=1)[cols], test[cols]])
print(combined.shape)
print(combined.columns)

combined = pd.get_dummies(combined)
print(combined.shape)
print(combined.columns)

train_new = combined[combined['PhoneId'].isin(train_ids)]
test_new = combined[combined['PhoneId'].isin(test_ids)]

(460, 17)
Index(['PhoneId', 'SIM Slot(s)', 'RAM', 'Height', 'Num_cores',
       'Internal Memory', 'Brand', 'os_name', 'Weight', 'Screen Size', 'Sim1',
       'Processor_frequency', 'SIM 2', 'Screen to Body Ratio (calculated)',
       'Resolution', 'Capacity', 'Pixel Density'],
      dtype='object')
(460, 87)
Index(['PhoneId', 'RAM', 'Height', 'Internal Memory', 'Weight', 'Screen Size',
       'Processor_frequency', 'Screen to Body Ratio (calculated)',
       'Resolution', 'Capacity', 'Pixel Density',
       'SIM Slot(s)_Dual SIM, GSM+CDMA', 'SIM Slot(s)_Dual SIM, GSM+GSM',
       'SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE',
       'SIM Slot(s)_Single SIM, GSM', 'Num_cores_312', 'Num_cores_Deca',
       'Num_cores_Dual', 'Num_cores_Hexa', 'Num_cores_Octa', 'Num_cores_Other',
       'Num_cores_Quad', 'Num_cores_Tru-Octa', 'Brand_10.or', 'Brand_Apple',
       'Brand_Asus', 'Brand_Billion', 'Brand_Blackberry', 'Brand_Comio',
       'Brand_Coolpad', 'Brand_Do', 'Brand_Gionee', 'Brand_Googl

In [103]:
train_new = train_new.merge(train[['PhoneId', 'Rating']], on='PhoneId')

In [104]:
# check the number of features and data points in train
print("Number of data points in train: %d" % train_new.shape[0])
print("Number of features in train: %d" % train_new.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test_new.shape[0])
print("Number of features in test: %d" % test_new.shape[1])

Number of data points in train: 341
Number of features in train: 88
Number of data points in test: 119
Number of features in test: 87


In [105]:
train_new.head()

,PhoneId,RAM,Height,Internal Memory,Weight,Screen Size,Processor_frequency,Screen to Body Ratio (calculated),Resolution,Capacity,...,os_name_Tizen,os_name_iOS,Sim1_2G,Sim1_3G,Sim1_4G,SIM 2_2G,SIM 2_3G,SIM 2_4G,SIM 2_Other,Rating
0,0,4,157.9,64,182,6.26,1.8,80.68,20,4000,...,0,0,0,0,1,0,0,1,0,4.5
1,1,3,156.2,32,168,6.20,1.8,80.85,8,4230,...,0,0,0,0,1,0,0,1,0,4.5
2,2,3,157.0,32,168,6.30,2.1,83.68,25,3500,...,0,0,0,0,1,0,0,1,0,4.4
3,4,4,159.8,64,169,6.00,2.2,74.78,24,3300,...,0,0,0,0,1,0,0,1,0,4.3
4,5,4,160.4,64,175,6.50,2.2,84.23,16,3750,...,0,0,0,0,1,0,0,1,0,4.4


In [106]:
test_new.head()

,PhoneId,RAM,Height,Internal Memory,Weight,Screen Size,Processor_frequency,Screen to Body Ratio (calculated),Resolution,Capacity,...,os_name_Other,os_name_Tizen,os_name_iOS,Sim1_2G,Sim1_3G,Sim1_4G,SIM 2_2G,SIM 2_3G,SIM 2_4G,SIM 2_Other
0,3,2,156.2,16,168,6.20,1.8,80.85,5,4230,...,0,0,0,0,0,1,0,0,1,0
1,11,4,156.0,64,205,6.20,1.8,81.60,12,5000,...,0,0,0,0,0,1,0,0,1,0
2,13,6,156.7,64,169,6.30,2.0,83.84,25,3500,...,0,0,0,0,0,1,0,0,1,0
3,16,4,156.7,64,169,6.30,2.0,83.84,16,3500,...,0,0,0,0,0,1,0,0,1,0
4,19,4,158.6,64,181,5.99,1.8,77.43,20,4000,...,0,0,0,0,0,1,0,0,1,0


## Dummy Solution

In [107]:
# submission = pd.DataFrame({'PhoneId':test_new['PhoneId'], 'Class':[1]*test_new.shape[0]})
# submission = submission[['PhoneId', 'Class']]
# submission.head()

In [108]:
# submission.to_csv("submission.csv", index=False)

In [109]:
train_new.head()

,PhoneId,RAM,Height,Internal Memory,Weight,Screen Size,Processor_frequency,Screen to Body Ratio (calculated),Resolution,Capacity,...,os_name_Tizen,os_name_iOS,Sim1_2G,Sim1_3G,Sim1_4G,SIM 2_2G,SIM 2_3G,SIM 2_4G,SIM 2_Other,Rating
0,0,4,157.9,64,182,6.26,1.8,80.68,20,4000,...,0,0,0,0,1,0,0,1,0,4.5
1,1,3,156.2,32,168,6.20,1.8,80.85,8,4230,...,0,0,0,0,1,0,0,1,0,4.5
2,2,3,157.0,32,168,6.30,2.1,83.68,25,3500,...,0,0,0,0,1,0,0,1,0,4.4
3,4,4,159.8,64,169,6.00,2.2,74.78,24,3300,...,0,0,0,0,1,0,0,1,0,4.3
4,5,4,160.4,64,175,6.50,2.2,84.23,16,3750,...,0,0,0,0,1,0,0,1,0,4.4


In [110]:
# Last column contains the ratings i.e. the label
print(train_new.shape, test_new.shape)

(341, 88) (119, 87)


In [111]:
# Labels have not been binarized
train_new['Rating'][25:40]

25    4.3
26    4.3
27    4.6
28    4.4
29    4.5
30    3.9
31    4.2
32    4.3
33    4.2
34    4.3
35    4.3
36    4.2
37    4.4
38    4.1
39    3.8
Name: Rating, dtype: float64

In [112]:
# Binarize the labels
train_new['Rating'] = train_new['Rating'].apply(lambda x: 1 if x>=4 else 0)

In [113]:
train_new['Rating'][25:40]

25    1
26    1
27    1
28    1
29    1
30    0
31    1
32    1
33    1
34    1
35    1
36    1
37    1
38    1
39    0
Name: Rating, dtype: int64

## Removing extreme values

In [114]:
# print(train_new.shape)
# train_new_modi = train_new[train_new.RAM < 16]
# print(train_new_modi.shape)

### Split train_new into X and y.

In [115]:
y = train_new['Rating']
X = train_new.drop('Rating', axis = 1)  # Not inplace, train_new is not affected

In [141]:
# print(X.shape, y.shape)
# print(X.RAM.value_counts())

(333, 87) (333,)
3    99
4    94
2    67
6    34
1    29
8    10
Name: RAM, dtype: int64


In [117]:
# Verifying that y contains the labels. Also data imbalance is observed hence stratified split is necessary in next step
y.value_counts()

1    235
0     98
Name: Rating, dtype: int64

In [118]:
# Split the training data into train and val set

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state = 42, stratify = y, test_size = 0.1)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)
print(y_train.mean(), y_val.mean())

(299, 87) (34, 87) (299,) (34,)
0.705685618729097 0.7058823529411765


## Normalizing and Standardizing the data

In [119]:
# Normalization/STandardization should not be done for one hot encoded columns
not_onehot = list()
for i in X_train.columns:
    if X_train[i].dtype != 'uint8':
        not_onehot.append(i)

In [120]:
# Normalizing
# X_train[not_onehot]=(X_train[not_onehot]-X_train[not_onehot].min())/(X_train[not_onehot].max()-X_train[not_onehot].min())
# X_val[not_onehot]=(X_val[not_onehot]-X_val[not_onehot].min())/(X_val[not_onehot].max()-X_val[not_onehot].min())
# (0.8627450980392157, 0.8, 10000, 0.001) ; LB = 0.77142


# Standardizing
X_train[not_onehot]=(X_train[not_onehot]-X_train[not_onehot].median())/X_train[not_onehot].std()
X_val[not_onehot]=(X_val[not_onehot]-X_val[not_onehot].median())/X_val[not_onehot].std()
## 0.869281045751634, 0.7714285714285715, 10000, 0.1 ; LB = 0.8


# X_train[not_onehot]=(X_train[not_onehot]-X_train[not_onehot].mean())/X_train[not_onehot].std()
# X_val[not_onehot]=(X_val[not_onehot]-X_val[not_onehot].mean())/X_val[not_onehot].std()
## (0.869281045751634, 0.7428571428571429, 5000, 0.01)

In [121]:
X_train.head(10)

,PhoneId,RAM,Height,Internal Memory,Weight,Screen Size,Processor_frequency,Screen to Body Ratio (calculated),Resolution,Capacity,...,os_name_Other,os_name_Tizen,os_name_iOS,Sim1_2G,Sim1_3G,Sim1_4G,SIM 2_2G,SIM 2_3G,SIM 2_4G,SIM 2_Other
163,-0.088333,0.000000,-0.412410,0.000000,0.000000,-0.080318,-0.67254,0.070950,-0.508970,0.985792,...,0,0,0,0,0,1,0,0,1,0
262,0.964299,0.654948,0.176747,0.669626,-0.377863,0.000000,-0.89672,-0.147061,1.357253,0.000000,...,0,0,0,0,0,1,0,0,1,0
19,-1.509017,0.000000,0.813037,0.000000,-0.047233,1.220833,0.00000,1.050067,0.000000,0.985792,...,0,0,0,0,0,1,0,0,1,0
241,0.714023,-0.654948,-1.072267,-0.334813,-1.086357,-0.803180,-0.67254,-0.592114,-0.508970,-0.514326,...,0,0,0,0,0,1,0,1,0,0
318,1.538461,-1.309896,-2.215232,-0.502220,-1.086357,-1.606360,-1.56926,-1.167458,-0.508970,-1.157234,...,0,0,0,0,0,1,0,0,1,0
97,-0.765550,-0.654948,-0.412410,-0.334813,0.000000,-0.080318,-0.67254,0.070950,-0.508970,0.985792,...,0,0,0,0,0,1,0,0,1,0
186,0.132499,-0.654948,-0.742338,-0.334813,-0.236165,-0.803180,-0.89672,-0.844956,0.848283,-0.064291,...,0,0,0,0,0,1,0,0,1,0
194,0.250276,3.274739,0.777688,2.008879,1.133590,0.803180,1.23299,0.767555,0.000000,0.771489,...,0,0,0,0,0,1,0,0,1,0
332,1.656238,0.000000,0.329928,0.000000,-0.047233,0.000000,-0.89672,-0.272192,0.848283,-0.085721,...,0,0,0,0,0,1,0,0,1,0
40,-1.295546,0.654948,0.648073,0.669626,0.330630,1.285088,0.67254,1.442229,2.884162,0.450035,...,0,0,0,0,0,1,0,0,1,0


In [122]:
train_new.iloc[[39, 41, 26, 254, 295, 117, 1, 135, 58, 122], :]

,PhoneId,RAM,Height,Internal Memory,Weight,Screen Size,Processor_frequency,Screen to Body Ratio (calculated),Resolution,Capacity,...,os_name_Tizen,os_name_iOS,Sim1_2G,Sim1_3G,Sim1_4G,SIM 2_2G,SIM 2_3G,SIM 2_4G,SIM 2_Other,Rating
39,56,4,154.5,64,180,5.5,2.30,70.95,13,4000,...,0,0,0,0,1,0,0,1,0,0
41,58,6,158.1,64,189,6.2,2.70,83.99,8,3500,...,0,0,0,0,1,0,0,1,0,1
26,38,6,162.5,128,183,6.3,2.20,80.86,24,3800,...,0,0,0,0,1,0,0,1,0,1
254,351,3,154.4,32,175,5.7,1.40,75.21,8,4000,...,0,0,0,0,1,0,0,1,0,1
295,410,3,152.4,32,170,5.5,1.60,69.46,5,3000,...,0,0,0,0,1,0,1,0,0,0
117,157,4,153.3,64,153,5.5,1.50,72.17,16,3200,...,0,0,0,0,1,0,0,1,0,1
1,1,3,156.2,32,168,6.2,1.80,80.85,8,4230,...,0,0,0,0,1,0,0,1,0,1
135,184,6,162.9,64,188,6.0,1.95,75.32,16,4000,...,0,0,0,0,1,0,0,1,0,1
58,76,8,157.5,128,182,6.4,2.00,85.05,25,3500,...,0,0,0,0,1,0,0,1,0,1
122,162,2,138.4,64,148,4.7,2.39,65.23,7,1821,...,0,1,0,0,1,0,0,0,1,1


In [123]:
X_train.shape

(299, 87)

## Remove unneccessary one-hot columns

In [124]:
percentage = 0.005


print(X_train.shape, X_val.shape)
# (0.7843137254901961, 0.7428571428571429, 5000, 1) : 5 percent
# (0.8529411764705882, 0.7714285714285715, 10000, 0.1) : 1 percent
# (0.8758169934640523, 0.8, 10000, 0.01) : 0.5 percent -----> LB score : 0.88571: columns reduced from 87 to 66
one_hot_drop = list()
for i in X_train.columns:
    if X_train[i].dtype == 'uint8':
        if X_train[i].shape[0]*percentage > X_train[i].sum(axis=0):
            one_hot_drop.append(i)

X_train.drop(one_hot_drop, axis = 1, inplace=True)
X_val.drop(one_hot_drop, axis = 1, inplace=True)

X_train.drop('PhoneId', axis = 1, inplace=True)
X_val.drop('PhoneId', axis = 1, inplace=True)

print(X_train.shape, X_val.shape)

(299, 87) (34, 87)
(299, 65) (34, 65)


## Perceptron Class

In [125]:
class Perceptron:
  
    def __init__ (self):
        self.w = None
        self.b = None

    def model(self, x):
        return 1 if (np.dot(self.w, x) >= self.b) else 0

    def predict(self, X):
        Y = []
        for x in X:
            result = self.model(x)
            Y.append(result)
        return np.array(Y)

    def fit(self, X, Y, epochs = 1, lr = 1):

        self.b = 0
        self.w = np.ones(X.shape[1])
        accuracy = {}
        max_accuracy = 0

#         wt_matrix = []

        for i in range(epochs):
            for x, y in zip(X, Y):
                y_pred = self.model(x)
                if y == 1 and y_pred == 0:
                    self.w = self.w + lr * x
                    self.b = self.b - lr * 1
                elif y == 0 and y_pred == 1:
                    self.w = self.w - lr * x
                    self.b = self.b + lr * 1

#             wt_matrix.append(self.w)    

            accuracy[i] = accuracy_score(self.predict(X), Y)
            if (accuracy[i] > max_accuracy):
                max_accuracy = accuracy[i]
                chkptw = self.w
                chkptb = self.b

        self.w = chkptw
        self.b = chkptb

        print('Max training accuracy:',max_accuracy)

#         plt.plot(accuracy.values())
#         plt.ylim([0, 1])
#         plt.show()

        return max_accuracy, (chkptw, chkptb)

## LR and Epoch grid search

In [126]:
# val_accs = list()
# # epochs = np.linspace(7500, 15000, num = 5, dtype = int)
# # LR = np.linspace(0.001, 0.1, num = 10)
# # (0.8725490196078431, 0.8285714285714286, 7500, 0.067) : LB- 0.85714

# # epochs = np.linspace(5000, 10000, num = 5, dtype = int)
# # LR = [0.001, 0.01, 0.05, 0.067, 0.1]

# epochs = np.linspace(5000, 10000, num = 2, dtype = int)
# LR = np.linspace(0.005, 0.05, num = 10)


# for i, j in list(itertools.product(epochs, LR)):
#     perceptron = Perceptron()
#     print('epochs:', i, ' ,LR:', j)
    
#     train_score, wt_matrix = perceptron.fit(X_train.values, y_train.values, i, j)
    
#     Y_pred_test = perceptron.predict(X_val.values)
    
#     val_score = accuracy_score(Y_pred_test, y_val.values)
#     print('Max Validation accuracy:', val_score)
#     print('-'*50)
#     val_accs.append((train_score, val_score, i, j, wt_matrix))

    
# max_val = 0
# Final_config = None
# for i in val_accs:
#     if i[1]>max_val:
#         Final_config = i
#         max_val = i[1]

# print(Final_config[:-1])
    

## LR uniformly decrease

In [127]:
# val_accs = list()
# epochs = [7500]
# LR = np.linspace(0.001, 0.1, num = 10)[::-1]

# perceptron = Perceptron()
# for i, j in list(itertools.product(epochs, LR)):
    
#     print('epochs:', i, ' ,LR:', j)
    
#     train_score, wt_matrix = perceptron.fit(X_train.values, y_train.values, i, j)
    
#     Y_pred_test = perceptron.predict(X_val.values)
    
#     val_score = accuracy_score(Y_pred_test, y_val.values)
#     print('Max Validation accuracy:', val_score)
#     print('-'*50)
#     val_accs.append((train_score, val_score, i, j, wt_matrix))

    
# max_val = 0
# Final_config = None
# for i in val_accs:
#     if i[1]>max_val:
#         Final_config = i
#         max_val = i[1]

# print(Final_config[:-1])
    

## Final model

In [128]:
perceptron = Perceptron()
train_score, wt_matrix = perceptron.fit(X_train.values, y_train.values, 10000, 0.01)
Y_pred_test = perceptron.predict(X_val.values) 
val_score = accuracy_score(Y_pred_test, y_val.values)
print('Max Validation accuracy:', val_score)

Max training accuracy: 0.8662207357859532
Max Validation accuracy: 0.8235294117647058


In [129]:
print(wt_matrix[0])
print(wt_matrix[1])

[ 4.47940737e-03  3.38625578e-02 -4.46170185e-02 -7.73826102e-02
  5.24084087e-02  3.66985478e-02 -4.61323718e-02 -2.67047541e-03
  6.56299528e-02 -7.07905034e-03 -7.52869989e-16  6.00000000e-02
  3.60000000e-01 -1.80000000e-01  5.70000000e-01  3.20000000e-01
  1.40000000e-01  8.30000000e-01  1.50000000e-01 -2.10000000e-01
  1.05000000e+00 -3.00000000e-02 -7.00000000e-02 -1.00000000e-01
  1.00000000e-01  1.00000000e+00 -1.30000000e-01  6.00000000e-02
  1.04000000e+00 -1.20000000e-01 -1.00000000e-01 -2.20000000e-01
  1.60000000e-01 -2.00000000e-01 -2.00000000e-02  2.00000000e-02
  1.00000000e-02  1.04000000e+00 -2.00000000e-02 -4.00000000e-02
  3.00000000e-02 -7.45931095e-16  1.15000000e+00  1.12000000e+00
  7.00000000e-02  3.00000000e-02  1.03000000e+00  1.80000000e-01
  9.00000000e-02  1.40000000e-01  2.00000000e-01  9.00000000e-02
  1.00000000e+00 -9.00000000e-02  4.30000000e-01  9.00000000e-02
 -5.30000000e-01  1.05000000e+00  8.30000000e-01  2.60000000e-01
  3.10000000e-01  7.00000

In [130]:
# for i, j in enumerate(list(pd.Series(list([wt_matrix[0]])).apply(pd.cut, bins=2, labels=[0, 1]))):
#     print(i, j)
weights = [i for i in wt_matrix[0] if np.median(wt_matrix[0])>i]

In [131]:
weights

[0.004479407370641155,
 0.03386255783182299,
 -0.044617018461228765,
 -0.0773826101893785,
 0.052408408653153715,
 0.03669854779186149,
 -0.046132371847222214,
 -0.002670475405922485,
 -0.0070790503371290425,
 -7.528699885739343e-16,
 0.059999999999999255,
 -0.18000000000000077,
 -0.2100000000000008,
 -0.030000000000000755,
 -0.07000000000000076,
 -0.10000000000000074,
 -0.13000000000000073,
 0.059999999999999255,
 -0.12000000000000073,
 -0.10000000000000074,
 -0.2200000000000008,
 -0.2000000000000008,
 -0.020000000000000746,
 0.019999999999999248,
 0.009999999999999247,
 -0.020000000000000746,
 -0.04000000000000076,
 0.02999999999999925,
 -7.4593109467002705e-16,
 0.02999999999999925,
 -0.09000000000000075,
 -0.530000000000001]

## Submission

In [132]:
test_new.head()

,PhoneId,RAM,Height,Internal Memory,Weight,Screen Size,Processor_frequency,Screen to Body Ratio (calculated),Resolution,Capacity,...,os_name_Other,os_name_Tizen,os_name_iOS,Sim1_2G,Sim1_3G,Sim1_4G,SIM 2_2G,SIM 2_3G,SIM 2_4G,SIM 2_Other
0,3,2,156.2,16,168,6.20,1.8,80.85,5,4230,...,0,0,0,0,0,1,0,0,1,0
1,11,4,156.0,64,205,6.20,1.8,81.60,12,5000,...,0,0,0,0,0,1,0,0,1,0
2,13,6,156.7,64,169,6.30,2.0,83.84,25,3500,...,0,0,0,0,0,1,0,0,1,0
3,16,4,156.7,64,169,6.30,2.0,83.84,16,3500,...,0,0,0,0,0,1,0,0,1,0
4,19,4,158.6,64,181,5.99,1.8,77.43,20,4000,...,0,0,0,0,0,1,0,0,1,0


In [133]:
# Submission_median.csv     --> LB 0.80
# temp = test_new['PhoneId']
# test_new[not_onehot]=(test_new[not_onehot]-test_new[not_onehot].median())/test_new[not_onehot].std()
# test_new.drop(one_hot_drop, axis = 1, inplace=True)
# test_new.drop('PhoneId', axis = 1, inplace=True)
# ------------------------------------------------------------

# Submission_min_max.csv      -->  LB 0.77142
# temp = test_new['PhoneId']
# test_new[not_onehot]=(test_new[not_onehot]-test_new[not_onehot].min())/(test_new[not_onehot].max()-test_new[not_onehot].min())
# test_new.drop(one_hot_drop, axis = 1, inplace=True)
# test_new.drop('PhoneId', axis = 1, inplace=True)

## Submission_dropcol0.005_median    --> LB 0.88571
## drop columns percentage = 0.005, dropped phoneid, epochs = 10K , LR = 0.01, median normalization
temp = test_new['PhoneId']
test_new[not_onehot]=(test_new[not_onehot]-test_new[not_onehot].median())/test_new[not_onehot].std()
test_new.drop(one_hot_drop, axis = 1, inplace=True)
test_new.drop('PhoneId', axis = 1, inplace=True)

# Submission_dropcol0.005_median_grid    -->   LB 0.85714
# temp = test_new['PhoneId']
# test_new[not_onehot]=(test_new[not_onehot]-test_new[not_onehot].median())/test_new[not_onehot].std()
# test_new.drop(one_hot_drop, axis = 1, inplace=True)
# test_new.drop('PhoneId', axis = 1, inplace=True)

# Submission_dropcol0.005_median_RAM   -->  LB 0.85714
# (0.862876254180602, 0.8529411764705882, 5000, 0.01)
# temp = test_new['PhoneId']

# test_new[not_onehot]=(test_new[not_onehot]-test_new[not_onehot].median())/test_new[not_onehot].std()
# test_new.drop(one_hot_drop, axis = 1, inplace=True)
# test_new.drop('PhoneId', axis = 1, inplace=True)

In [134]:
# # Predict using model
test_new_preds = perceptron.predict(test_new.values)

# Create dataframe
submission = pd.DataFrame({'PhoneId':temp, 'Class':test_new_preds})
submission = submission[['PhoneId', 'Class']]
submission.head()

# Write to csv file
submission.to_csv("submission_verify.csv", index=False)

### Compare how different predictions too existing ones

In [135]:
a = pd.read_csv('submission_verify.csv')
b = pd.read_csv('submission_dropcol0.005_median.csv')
count = 0
for i, j in zip(a.Class.values,b.Class.values):
    if i!= j:
        count += 1
print('Differ by:', count)

Differ by: 14


## Ensemble

In [136]:
# import math

# count = 0
# # for i in os.listdir():
#     # if 'submission_dro' in i:
# for i in ['submission_dropcol0.005_median_RAM.csv', 'submission_dropcol0.005_median.csv', 'submission_dropcol0.005_median_grid.csv']:
#     if count == 0:
#         tot = pd.read_csv(i)
#         count+=1
#     else: 
#         tot += pd.read_csv(i)
#         count += 1

# tot = tot.applymap(lambda x: math.ceil(x/2))
# # tot = tot.applymap(lambda x: round(x/3))
# tot.to_csv("EofE_dropcols_dropcolsRAM_0.88_088.csv", index=False)
# tot.head(10)

In [137]:
# train_new.groupby('Rating')['RAM'].describe()

# train_new.RAM.unique()

# train_new.RAM.value_counts()

# print(test_new.shape)
# test_new = test_new[test_new.RAM < 16]
# print(test_new.shape)